In [1]:
import ipywidgets as widgets

import transformers
import torch

## Configuration

In [2]:

widgets.Dropdown(
    options=['English', 'Chinese'],
    value='Chinese',
    description='Language to use for topic extraction:',
    disabled=False,
)

Dropdown(description='Language to use for topic extraction:', index=1, options=('English', 'Chinese'), value='…

In [3]:
import pandas as pd
from datasets import Dataset, Features, Value, ClassLabel

In [4]:
def load_dataset(sources=['JOIN Proposals', 'iVoting Proposals'], text_columns=['title', 'proposal'], language='en'):
    all_data = pd.read_excel('data/JOIN_iVoting_Proposals_categorized.xlsx')
    all_data['label'] = all_data['Category']
    training_dataset = pd.DataFrame(data={
        'text': all_data[[column + '_' + language for column in text_columns]].apply(lambda row: '_'.join(row.values.astype(str)), axis=1),
        'label': all_data['Category']
    })
    # check for int in text column
    return Dataset.from_pandas(training_dataset, features=Features({'text': Value('string'), 'label': ClassLabel(names=all_data['Category'].unique().tolist())}))

dataset = load_dataset()

In [5]:
from torch.utils.data import DataLoader
dataset.set_format(type="torch", columns=["text", "label"])
dataloader = DataLoader(dataset, batch_size=1)
    

In [6]:
from transformers import AutoTokenizer
import transformers
import torch

model = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device="auto")

/home/korjakow/.cache/pypoetry/virtualenvs/03-urbandevelopmenttaiwan-HQCD10_a-py3.11/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/home/korjakow/.cache/pypoetry/virtualenvs/03-urbandevelopmenttaiwan-HQCD10_a-py3.11/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sente

RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, ort, xla, lazy, vulkan, mps, meta, hpu, mtia, privateuseone device type at start of device string: auto

In [ ]:
dataset.features["label"].names

[nan,
 'Law & Justice ',
 'Transport',
 'Social ',
 'Military and Natonal Security ',
 'Environment & Climate',
 'Finance ',
 'Education ',
 'Energy ']

In [7]:
for batch in dataloader:
    sequences = pipeline(
        batch["text"],
        candidate_labels=[
 'Law & Justice ',
 'Transport',
 'Social ',
 'Military and National Security ',
 'Environment & Climate',
 'Finance ',
 'Education ',
 'Energy '],
    )
    print(sequences[0]['sequence'])
    print(sequences[0]['labels'])
    print(sequences[0]['scores'])

NameError: name 'pipeline' is not defined